# Introduction to jfk-fling

``jfk-fling`` is a Jupyter Kernel for the Fortran language, specifically designed for learning the Fortran language.

The kernel allows you to execute code cells interactively and iteratively. The code to execute is parsed and then put into a form that is compiled and executed, with the return output being shown inline.

For example, let's run a hello world print statement in Fortran.

In [ ]:
print *, "Hello from jfk-fling!"

jfk-fling works by aggregating all previously executed cells so that you can iteratively develop code in a natural fashion.

For example, let's define a variable in one cell and use it in another:

In [ ]:
character (len = 40) :: message

message = "Hello from jfk-fling"

In [ ]:
print *, message

We can define subroutines:

In [ ]:
subroutine my_subroutine()
    print *, "Hello from my_subroutine"
end subroutine my_subroutine


call my_subroutine()

And functions:

In [ ]:
function my_function()
    print *, "Hello from my function"
    my_function = 42
end function my_function


result = my_function()

## Important implementation detail regarding jfk-fling

Whilst it may appear that cells are being executed iteratively, each cell is in fact an entirely independent execution environment with each cell being gathered together cumulatively.

This has two major implications:

 1. Run time will typically *always* increase as more cells are executed.
 2. You can't re-define previously defined entities.
 
Let's take a look at the code that we have aggregated so far:

In [ ]:
%code

As you can see, we are re-running **all** of the code that has previously been written in the cells, but are applying a few tricks (like stdout/stderr redirection) to make it look like only the last cell is being executed.

If at any point you wish to wipe the cells that have been accumulated, the ``%clear`` magic can be used.

In [ ]:
%clear

We can see that the current code is now in the most basic form possible:

In [ ]:
%code

# Choice of source form

jfk-fling supports a number of source forms. We have already seen the simplest form: a collection of Fortran statements such as ``print *, "Hello world!"``. Additionally we may issue simple declarations such as ``integer :: my_int`` as well as subroutine and function definitions.

Going one step further, complete program and module definitions are supported:

In [ ]:
MODULE my_module
    REAL :: pi = 3.14
END MODULE my_module

In [ ]:
PROGRAM my_program
    USE my_module
    print *, "Pi: ", pi
END PROGRAM my_program

# Error handling

Errors may come from a number of sources with jfk-fling:

1. Invalid syntax

In [ ]:
This is invalid Fortran syntax

2. Redefinition of entities

In [ ]:
INTEGER :: my_number

In [ ]:
REAL :: my_number

3. Runtime errors (e.g Segmentation Fault)

...

In all scenarios, jfk-fling will drop the offending cell in subsequent cell accumulations.

The exception to this is the STOP statement, which persists:

In [ ]:
STOP 'Error!'

In [ ]:
print*, 'Hello'

In [ ]:
%clear

## Fragments

Sometimes it is useful to separate blocks of logically consistent code into separate cells. For this the ``%%fragment`` magic should be used.
For example the definition of an object must occur at the same time as its methods, but sometimes it is nice to separate these into distinct cells:

In [ ]:
%%fragment

MODULE aclass_module

TYPE AClass
    INTEGER :: an_attribute
  CONTAINS
    PROCEDURE :: a_method
END TYPE AClass

In [ ]:
! Here we continue the aclass_module definition...

CONTAINS
    SUBROUTINE a_method(this, an_attribute)
        class(AClass) :: this
        integer :: an_attribute

        this%an_attribute = an_attribute
    END SUBROUTINE a_method

END MODULE aclass_module

In [ ]:
use aclass_module

TYPE(AClass) :: an_instance

call an_instance%a_method(10)

print *, an_instance%an_attribute

In [ ]:
%code